# Algoritmo A*
# Problema: O Caixeiro Viajante

# 1. Código

In [ ]:
# Importa biblioteca auxiliar para visualizar os resultados
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
import random
import math
import heapq

# Função para calcular a distância entre duas cidades
def distancia(cidade1, cidade2):
    return math.sqrt((cidade1[0] - cidade2[0])**2 + (cidade1[1] - cidade2[1])**2)

# Função para ler as coordenadas das cidades
# Retorna as coordenadas das cidades e os pesos (distâncias) entre cada duas cidades
def leitura():
    fin = open("entrada.txt", "r") # abre o arquivo de leitura
    x = list(map(float, fin.readline().split())) # guarda as coordenadas x das cidades
    y = list(map(float, fin.readline().split())) # guarda as coordenadas y das cidades
    fin.close() # fecha o arquivo de leitura
    cidades = [ (x[i], y[i])  for i in range (len(x))] # organiza as cidades lidas
    pesos = []
    # calcula os pesos (distâncias) das cidades
    for cidade1 in cidades:
        coluna = []
        for cidade2 in cidades:
            coluna.append(distancia(cidade1, cidade2))
        pesos.append(coluna)
    return cidades, pesos

# Função auxiliar para encontrar a raiz de uma cidade e retorná-la
def encontrar(subconjuntos, cidade):
    if subconjuntos[cidade] == -1:
        return cidade
    return encontrar(subconjuntos, subconjuntos[cidade])

# Função auxiliar que liga a raiz de duas cidades
def unir(subconjuntos, cidade1, cidade2):
    raizCidade1 = encontrar(subconjuntos, cidade1)
    raizCidade2 = encontrar(subconjuntos, cidade2)
    subconjuntos[raizCidade1] = raizCidade2

# função heurística do custo da árvore geradora mínima
# a função por default não considera o retorno para a cidade de início do circuito,
# por isso, pode-se decidir deixando o parâmetro comPartida como True
def h(caminho, numero_cidades, pesos, comPartida):
    ultimo = caminho[len(caminho) - 1] # pega a ultima cidade do caminho

    abertos = [] # lista de cidades disponíveis para visitar
    # caso a variação permitir a cidade de partida no cálculo da estimativa, adiciona à lista de abertos
    if comPartida:
        abertos.append(ultimo)
    # determina quais as cidades disponíveis (as que não estão no caminho)
    for cidade in range(numero_cidades):
        if cidade not in caminho:
            abertos.append(cidade)

    custo = 0
    heap_aux = [] # criando heap para ordenar os pesos
    # organiza em uma heap min os pesos de uma cidade para outra
    for cidade1 in abertos:
        for cidade2 in abertos:
            if (cidade1 != cidade2):
                # insere somente os pesos das cidades destino que ainda estão disponíveis para visitar
                heapq.heappush(heap_aux,(pesos[cidade1][cidade2], (cidade1, cidade2)))

    # cria matriz zerada para representar a árvore geradora mínima
    # se celula continuar 0, não há uma aresta entre as cidades (linha = cidade de origem, coluna = cidade de destino)
    agm = []
    for _ in range(numero_cidades):
        vetor = [0] * numero_cidades
        agm.append(vetor)

    # cria vetor que organiza as cidades interligadas
    subconjuntos = [-1] * numero_cidades

    while len(heap_aux) > 0:
        peso, aresta = heapq.heappop(heap_aux) # pega a menor aresta
        raizCidade1 = encontrar(subconjuntos, aresta[0]) # encontra a raiz
        raizCidade2 = encontrar(subconjuntos, aresta[1]) # encontra a raiz

        if raizCidade1 != raizCidade2: # adiciona à AGM o peso das cidades não interligadas
            agm[aresta[0]][aresta[1]] = peso
            unir(subconjuntos, raizCidade1, raizCidade2) # une as cidades

    # soma as arestas da AGM
    for linha in agm:
        custo += sum(linha)

    return custo

# função que calcula custo real do estado inicial até n
def g(caminho, pesos, numero_cidades):
    custo = 0
    for cidade in range(len(caminho) - 1): # percorre todas as cidades somando as distâncias de uma para outra
        custo += pesos[caminho[cidade]][caminho[cidade + 1]]
    # caso todas as cidades tenham sido percorridas, fecha o ciclo (soma o custo da última cidade com a primeira)
    if (len(caminho) == numero_cidades):
        custo += pesos[caminho[numero_cidades - 1]][caminho[0]] # fecha o ciclo
    return custo

# Classe que guarda as informações de um caminho
class Caminho:
    def __init__(self, caminho) -> None:
        self.caminho = caminho # caminho de cidades percorridas
        self.g = 0 # custo da função g
        self.h = 0 # custo da função h
        self.f = self.g + self.h # custo da função f

    def calcularF(self):
        self.f = self.g + self.h # custo da função f

    def ultima(self): # retorna a última cidade do caminho
        return self.caminho[len(self.caminho) - 1]

# função para calcular o custo de um caminho
def funcaoCusto(caminhoLista, pesos, cidades, comPartida):
    caminho = Caminho(caminhoLista)
    caminho.g = g(caminho.caminho, pesos, len(cidades)) # armazena a função g (o custo real do caminho atual)

    # armazena a função f (o custo segundo a função heurística que calcula o custo de um tal nó até o objetivo)
    # a cidade de partida vai ser a ultima do caminho temporario, o objetivo é usar todas as cidades no novo circuito (usando todos, fecharemos um ciclo)
    caminho.h= h(caminho.caminho, len(cidades), pesos, comPartida)

    caminho.calcularF() # calcula o custo da função f
    return caminho

# função principal que põe em prática o algoritmo A*
# a função por default não considera o retorno para a cidade de início do circuito,
# por isso, pode-se decidir deixando o parâmetro comPartida como True
def a_star(cidades, pesos, cidade_inicial, comPartida):
    visitadas = 0 # guarda o número de cidades visitadas
    gerados = 0 # guarda o número de cidades geradas

    visitadas += 1 # incrementa cidade visitada
    circuito = Caminho([cidade_inicial]) # caminho atual (já com a cidade de origem)
    gerados += 1 # incrementa cidade gerada
    it = 0 # variavel para saber a iteração
    heap_min = []  # guarda os custos de todos os caminhos visitados (percorridos)
    #enquanto o circuito não tiver com todas as cidades, executa
    while len(circuito.caminho) < len(cidades):
        abertos = set()
        # guarda as cidades abertas (disponíveis para visita)
        for cidade in range(len(cidades)):
            if cidade not in circuito.caminho:
                abertos.add(cidade)
        # visita todas as cidades em aberto, calcula seus custos e adiciona na heap min
        for cidade in abertos:
            visitadas += 1 # adiciona 1 em cidades visitadas

            caminho_temp = Caminho(circuito.caminho + [cidade]) # armazena o caminho temporário

            caminho_temp.g = g(caminho_temp.caminho, pesos, len(cidades)) # armazena a função g (o custo real do caminho atual)

            # armazena a função f (o custo segundo a função heurística que calcula o custo de um tal nó até o objetivo)
            # a cidade de partida vai ser a ultima do caminho temporario, o objetivo é usar todas as cidades no novo circuito (usando todos, fecharemos um ciclo)
            caminho_temp.h= h(caminho_temp.caminho, len(cidades), pesos, comPartida)

            caminho_temp.calcularF() # calcula o custo da função f

            # adiciona o valor de f (g + h) na heap-min
            heapq.heappush(heap_min, (caminho_temp.f, caminho_temp.caminho))
            it += 1 # incrementa a iteração
        _, caminhoLista = heapq.heappop(heap_min) # pega o caminho de menor custo
        caminho = funcaoCusto(caminhoLista, pesos, cidades, comPartida)
        gerados += 1 # incrementa o valor de cidades geradas
        circuito = caminho # atualiza o valor do circuito com o caminho de menor custo

    return visitadas, gerados, circuito.caminho, circuito.f

- Organização de Cidades e Cálculo de pesos

In [ ]:
# Função para ler as coordenadas das cidades
def leitura(entrada_x, entrada_y):
    X = [float(x) for x in entrada_x]  # guarda os valores de x
    Y = [float(y) for y in entrada_y]   # guarda os valores de y
    cidades = [(X[i], Y[i]) for i in range(len(X))]  # organiza as cidades
    pesos = []
    # calcula os pesos (a distância) de uma cidade para outra
    for cidade1 in cidades:
        coluna = []
        for cidade2 in cidades:
            coluna.append(distancia(cidade1, cidade2))
        pesos.append(coluna)
    # retorna as coordenadas das cidades e os pesos de cada cidade
    return cidades, pesos

# 2. Bloco de Experimentação

## 2.1. Entradas: Coordenadas das cidades

In [ ]:
# Valores de x das cidades
entrada_x = ['0.0', '1.0', '2.0', '0.0', '1.0', '1.0', '3.0']
# Valores de y das cidades
entrada_y = ['0.0', '2.0', '0.0', '2.0', '1.0', '3.0', '2.0']

In [ ]:
# Valores de x das cidades
entrada_X = ['3.0', '1.2', '7.2', '9.0', '6.5', '2.3', '8.8', '4.6', '5.2', '1.8', '6.0', '3.7']
# Valores de y das cidades
entrada_Y = ['5.0', '8.6', '4.3', '2.0', '9.2', '1.7', '6.1', '3.5', '7.7', '4.0', '2.9', '8.4']

- Código de teste
- Parâmetros:
1. cidades - coordenadas (x,y) de cada cidade.
2. pesos - valor dos pesos de uma cidade para outra.
3. comPartida - se a cidade inicial será relevante na função heurística.
4. sorteadas - lista de cidades que foram sorteadas caso o número de cidades da entrada seja maior que 10.

In [32]:
def testes (cidades, pesos, comPartida, sorteadas = []):
    tamanho_cidades = len(cidades)
    # Questão 1: estado inicial arbritário ou não
    # se tiver mais de 10 cidades, é arbitrário, senão segue a ordem de cidades da entrada
    iniciais = []
    if (tamanho_cidades > 10):
        iniciais = sorteadas
    else:
      iniciais = list(range(tamanho_cidades))
    matriz = []
    columns = [['Cidade_Inicial', 'Versao_da_Heuristica', 'Visitadas', 'Geradas', 'Circuito', 'Custo']]
    for cidade in iniciais: # para cidade da entrada, chama o algoritmo passando ela como cidade inicial
      visitados, gerados, circuito, custo = a_star(cidades, pesos, cidade, comPartida)
      linha = []
      linha.append(cidade)
      linha.append(f"{comPartida}")
      linha.append(visitados)
      linha.append(gerados)
      linha.append(circuito)
      linha.append(custo)
      matriz.append(linha)
    df = pd.DataFrame(matriz, columns = columns)
    return df

## 2.2. Número de cidades de entrada menor ou igual a 10




- Leitura de cidades e pesos

In [33]:
cidades, pesos = leitura(entrada_x, entrada_y)

### 2.2.1. Variação com a cidade de partida na estimativa

- Execução do código

In [34]:
df = testes(cidades, pesos, True)
display(df)

,Cidade_Inicial,Versao_da_Heuristica,Visitadas,Geradas,Circuito,Custo
0,0,True,914,471,"[0, 4, 1, 3, 5, 6, 2]",11.300563
1,1,True,555,247,"[1, 3, 0, 4, 2, 6, 5]",11.300563
2,2,True,890,448,"[2, 0, 4, 1, 3, 5, 6]",11.300563
3,3,True,723,348,"[3, 0, 4, 2, 6, 5, 1]",11.300563
4,4,True,515,233,"[4, 0, 2, 6, 5, 3, 1]",11.300563
5,5,True,796,395,"[5, 1, 3, 0, 4, 2, 6]",11.300563
6,6,True,1022,532,"[6, 2, 0, 4, 1, 3, 5]",11.300563


- Média de cidades visitadas e geradas

In [35]:
df[['Visitadas', 'Geradas']].mean()

Visitadas    773.571429
Geradas      382.000000
dtype: float64

### 2.2.2. Variação sem a cidade de partida na estimativa.

- Execução do código

In [40]:
df = testes(cidades, pesos, False)
display(df)

,Cidade_Inicial,Versao_da_Heuristica,Visitadas,Geradas,Circuito,Custo
0,0,False,1586,950,"[0, 4, 1, 3, 5, 6, 2]",11.300563
1,1,False,1360,786,"[1, 3, 0, 4, 2, 6, 5]",11.300563
2,2,False,1596,959,"[2, 0, 4, 1, 3, 5, 6]",11.300563
3,3,False,1416,832,"[3, 0, 4, 2, 6, 5, 1]",11.300563
4,4,False,1295,745,"[4, 0, 2, 6, 5, 3, 1]",11.300563
5,5,False,1504,891,"[5, 1, 3, 0, 4, 2, 6]",11.300563
6,6,False,1649,994,"[6, 2, 0, 4, 1, 3, 5]",11.300563


- Média de cidades visitadas e geradas

In [41]:
df[['Visitadas', 'Geradas']].mean()

Visitadas    1486.571429
Geradas       879.571429
dtype: float64

## 2.3. Número de cidades de entrada maior que 10

- Leitura de cidades e pesos
- Sorteio de cidades iniciais

In [43]:
cidades, pesos = leitura(entrada_X, entrada_Y)
sorteadas = []
while(len(sorteadas) <= 10): #aciona elementos no set até o tamanho ser igual a 10
  cidade = random.choice(range(len(cidades))) # escolhe uma cidade aleatoria para ser a cidade inicial
  if cidade not in sorteadas:
    sorteadas.append(cidade)

### 2.3.1. Variação com a cidade de partida na estimativa.

- Execução do código

In [44]:
df = testes(cidades, pesos, True, sorteadas)
display(df)

,Cidade_Inicial,Versao_da_Heuristica,Visitadas,Geradas,Circuito,Custo
0,11,True,3893,783,"[11, 1, 0, 9, 5, 7, 10, 3, 2, 6, 4, 8]",30.853768
1,9,True,5005,1013,"[9, 5, 7, 10, 3, 2, 6, 4, 8, 11, 1, 0]",30.853768
2,5,True,8764,2021,"[5, 7, 10, 3, 2, 6, 4, 8, 11, 1, 0, 9]",30.853768
3,3,True,10175,2441,"[3, 10, 7, 5, 9, 0, 1, 11, 8, 4, 6, 2]",30.853768
4,8,True,4426,866,"[8, 11, 1, 0, 9, 5, 7, 10, 3, 2, 6, 4]",30.853768
5,10,True,3916,772,"[10, 7, 5, 9, 0, 1, 11, 8, 4, 6, 2, 3]",30.853768
6,2,True,4015,828,"[2, 3, 10, 7, 5, 9, 0, 1, 11, 8, 4, 6]",30.853768
7,7,True,3785,773,"[7, 5, 9, 0, 1, 11, 8, 4, 6, 2, 3, 10]",30.853768
8,1,True,9846,2432,"[1, 0, 9, 5, 7, 10, 3, 2, 6, 4, 8, 11]",30.853768
9,4,True,7523,1644,"[4, 6, 2, 3, 10, 7, 5, 9, 0, 1, 11, 8]",30.853768


- Média de cidades visitadas e geradas

In [45]:
df[['Visitadas', 'Geradas']].mean()

Visitadas    5948.000000
Geradas      1310.181818
dtype: float64

### 2.3.2. Variação sem a cidade de partida na estimativa.

- Execução do código

In [46]:
df = testes(cidades, pesos, False, sorteadas)
display(df)

,Cidade_Inicial,Versao_da_Heuristica,Visitadas,Geradas,Circuito,Custo
0,11,False,15325,3979,"[11, 1, 0, 9, 5, 7, 10, 3, 2, 6, 4, 8]",30.853768
1,9,False,20779,5232,"[9, 5, 7, 10, 3, 2, 6, 4, 8, 11, 1, 0]",30.853768
2,5,False,33020,9522,"[5, 7, 10, 3, 2, 6, 4, 8, 11, 1, 0, 9]",30.853768
3,3,False,37712,11233,"[3, 10, 7, 5, 9, 0, 1, 11, 8, 4, 6, 2]",30.853768
4,8,False,16247,4174,"[8, 11, 1, 0, 9, 5, 7, 10, 3, 2, 6, 4]",30.853768
5,10,False,16009,3990,"[10, 7, 5, 9, 0, 1, 11, 8, 4, 6, 2, 3]",30.853768
6,2,False,14641,3731,"[2, 3, 10, 7, 5, 9, 0, 1, 11, 8, 4, 6]",30.853768
7,7,False,14869,3743,"[7, 5, 9, 0, 1, 11, 8, 4, 6, 2, 3, 10]",30.853768
8,1,False,36948,11190,"[1, 0, 9, 5, 7, 10, 3, 2, 6, 4, 8, 11]",30.853768
9,4,False,26978,7548,"[4, 6, 2, 3, 10, 7, 5, 9, 0, 1, 11, 8]",30.853768


- Média de cidades visitadas e geradas

In [47]:
df[['Visitadas', 'Geradas']].mean()

Visitadas    22575.272727
Geradas       6216.545455
dtype: float64

# 3. Análise

De modo geral, todos os testes encontraram um caminho de menor custo, sendo ele distinto dos demais.

Sobre as cidades visitadas e geradas, as quantidades foram bem diferentes entre as duas variações:

- Com o teste de 7 cidades, na variação 1, a quantidade de cidades visitadas, em média, foi de 773.571429 e o de geradas foi 382. Na variação 2, o número de cidades visitadas foi de 1486.571429 e o de geradas foi de 879.571429. Houve um aumento de quase o dobro de cidades visitadas e geradas ao não incluir a cidade de partida no cálculo das estimativas.

- Com o teste de 12 cidades, na variação 1, a quantidade de cidades visitadas, em média, foi de 5948 e o de geradas foi de 1310.181818. Na variação 2, o número de cidades visitas foi, em média, 22575.272727 e o de geradas foi de 6216.545455. Houve um aumento maior que o dobro em comparação com a variação 1.